In [1]:
import os
import pandas as pd
import pydicom as dicom
import matplotlib.pylab as plt
import cv2
import numpy as np
from glob import glob
import time

In [2]:
base_directory = 'rsna-pneumonia-detection-challenge'

## Checking sample images

In [ ]:
image_train_path = os.listdir(base_directory + '/stage_2_train_images')
image_test_path = os.listdir(base_directory + '/stage_2_test_images')
print("#images in train set:", len(image_train_path))
print("#images in test set:", len(image_test_path))

In [ ]:
train_csv = pd.read_csv(base_directory + '/stage_2_train_labels.csv')
train_csv.head()

In [ ]:
train_csv['Target'].value_counts()

In [ ]:
image_name = '7be6b4de-afe9-43c0-a581-0f49608c8976'
# train_csv[train_csv['patientId'==image_name]]
train_csv[train_csv.isin([image_name]).any(axis=1)]

In [ ]:
id = 12059
image_name = train_csv['patientId'][id]+'.dcm'
x1 = int(train_csv['x'][id])
y1 = int(train_csv['y'][id])
width = int(train_csv['width'][id])
height = int(train_csv['height'][id])
label = train_csv['Target'][id]
print(label, image_name, x1, y1, width, height)

In [ ]:
x2 = int(x1 + width)
y2 = int(y1 + height)
x2, y2

In [ ]:
# specify your image path
path = 'rsna-pneumonia-detection-challenge/stage_2_train_images/'
image_path = path + image_name
ds = dicom.dcmread(image_path)

image_data = ds.pixel_array
plt.figure(figsize=(7,7))
plt.imshow(image_data, cmap = 'gray')

# plt.show()

In [ ]:
image_data.shape

In [ ]:
cv2.rectangle(image_data, (x1, y1), (x2, y2), (255,0,0), 2)
plt.figure(figsize=(7,7))
plt.imshow(image_data, cmap = 'gray')

# Convert to JPG and duplicating channels

In [3]:
all_images = glob(os.path.join(base_directory, 'stage_2_test_images', '*.dcm'))
len(all_images), all_images[0]

(3000,
 'rsna-pneumonia-detection-challenge/stage_2_test_images/2392af63-9496-4e72-b348-9276432fd797.dcm')

In [5]:
sizes = []
for i, image_path in enumerate(all_images):
    
    name = os.path.splitext(image_path.split('/')[-1])[0]
#     print(i, name)
    
    ds = dicom.dcmread(image_path)
    image_data = ds.pixel_array
    
    image_data_3 = np.dstack((image_data, image_data, image_data))
    image_data_3_resized = cv2.resize(image_data_3, (512,512))
    
    save_path = os.path.join(base_directory, 'test_jpgs', name+'.jpg')
    cv2.imwrite(save_path, image_data_3_resized)
    
    sizes.append(image_data.shape)
    
    c = 1000
    if i%c == 0 and i>0:
        print(i/c)
        time.sleep(60)

In [6]:
len(sizes), list(set(sizes))

(3000, [(1024, 1024)])